# BERT의 모든 인코더 레이어에서 임베딩 추출하기
지난 장에서 사전 학습된 BERT로부터 임베딩을 추출하는 방법에 대해 알아 보았습니다. 마지막 인더 레이어로부터 임베딩을 추출한다고 배웠습니다. 이제 생기는 의문은 오직 마지막 인코더 레이어(마지막 은닉 상태)로부터 얻은 임베딩만을 고려해야 될까요? 아니면, 모든 인코더 레이어(모든 은닉 상태)로부터 얻은 임베딩을 고려해야 될까요? 이 점에 대해 좀 더 살펴보겠습니다.

아래 그림과 같이, 입력 임베딩 레이어를 $h_0$라고 하고 첫번째 레이어(첫번째 은닉층)을 $h_1$, 두번째 인코더 레이어(두번째 은닉층)를 $h_2$이라 하고 계속 진행하여 마지막 12번째 인코더 레이어를 $h_{12}$라고 표시하겠습니다:


![title](images/4.png)


마지막 인코더 레이어로부터만 임베딩(표현)을 가져오는 대신에, BERT의 저자는 다른 인코더 레이어들로부터 임베딩을 가져오는 실험을 하였습니다.

예를 들어, named-entity recognition task에서 저자는 사전 학습된 BERT를 특징을 추출하는데 사용하였습니다. 마지막 인코더 레이어(마지막 은닉층)로부터 얻은 임베딩을 특징(feature)으로 사용하는 대신에, 다른 인코더 레이어들(다른 은닉층들)로부터 얻은 임베딩을 특징으로 사용하였고 아래의 F1 점수를 얻었습니다:


![title](images/5.png)

위 테이블에서 알 수 있듯이, 마지막 4개의 인코더 레이어(마지막 4개의 은닉층)으로부터 얻은 임베딩을 연쇄적으로 잇는 것이 더 좋은 F1 점수(96점)를 보여줬습니다. 1% in the NER task. 그러므로, 마지막 인코더 레이어(마지막 은닉층)로부터만 임베딩을 얻기보다 다른 인코더 레이어들로부터 얻은 임베딩들 역시 사용할 수 있음을 알 수 있습니다.

이제, 트랜스포머 라이브러리를 활용하여 모든 인코더 레이어로부터 임베딩을 추출하는 방법에 대해 알아보겠습니다.

## 임베딩 추출하기
첫째로, 필요한 모듈을 불러 옵니다:

In [ ]:
%%capture 
#!pip install transformers==3.5.1
!pip install transformers

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

이제, 사전 학습된 BERT 모델과 토큰나이저를 다운로드 합니다. 사전 학습된 BERT 모델을 다운로드하면서 확인할 수 있듯이, output_hidden_state를 True로 설정해야 됩니다. 이렇게 설정함으로써, 모든 인코더 레이어로부터 임베딩들을 가져올 수 있습니다:

<font color="blue">[역자]</font>

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

다음으로, 모델에 입력을 넣기 전에 입력 데이터를 전처리합니다.

## 입력 전처리하기
이전 장에서 본 것과 같은 문장으로 진행합니다. 먼저, 문장을 토큰화하고 [CLS] 토큰을 문장의 시작에 그리고 [SEP] 토큰을 문장의 끝에 추가합니다:


In [ ]:
sentence = 'I love Paris'
tokens = tokenizer.tokenize(sentence)
tokens = ['[CLS]'] + tokens + ['[SEP]']

토큰의 전체 길이를 7로 유지한다고 가정합니다. 따라서, [PAD] 토큰을 추가해야되고 attention mask 또한 정의해야 됩니다:

In [ ]:
tokens = tokens + ['[PAD]'] + ['[PAD]']
attention_mask = [1 if i!= '[PAD]' else 0 for i in tokens]

다음으로, tokens를 token_ids로 변환합니다:

In [ ]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)

이제, token_ids와 attention_mask를 텐서로 변환합니다:

In [ ]:
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

다음으로 입력 데이터를 전처리하여, 임베딩을 얻습니다.

## 임베딩 얻기
모델의 모든 인코더 레이어로부터 임베딩을 얻도록 모델을 정의하면서 output_hidden_states를 True로 설정하였기에, 아래에서 보는 바와 같이 모델은 세가지 값을 출력값으로 반환합니다

In [ ]:
last_hidden_state, pooler_output, hidden_states = model(token_ids, attention_mask = attention_mask)

위 코드를 자세히 살펴보면 아래와 같습니다:

첫번째 값인 last_hidden_state는 오직 마지막 인코더 레이어(인코더 12)로부터 얻은 모든 토큰의 표현(<font color="blu">[역자]</font>임베딩 표현. 이 예제에선 사이즈가 (1,512,7))을 포함합니다. 
다음으로, pooler_output은 선형 변환과 tanh 활성화함수에 의해 후처리된 마지막 인코더 레이어로부터 얻은 [CLS] 토큰에 대한 표현을 의미합니다.
hidden_states는 모든 인코더 레이어의 마지막 레이어로부터 얻은 모든 토큰에 대한 표현을 포함합니다(<font color="blu">[역자]</font>즉, 각 인코더 레이어의 임베딩을 하나로 묶은 것 입니다)
이제, 각각의 값들을 살펴보고 자세히 알아보겠습니다.

last_hidden_state 먼저 살펴보겠습니다. 앞서 말한 바와 같이, 이 반환값은 오직 마지막 인코더 레이어(인코더 1)에서 얻은 모든 토큰의 표현을 갖고 있습니다. last_hidden_state의 사이즈를 출력해봅니다:

In [ ]:
last_hidden_state.shape

torch.Size([1, 7, 768])

사이즈 [1, 7, 768]은 각각 [batch_size, sequence_length, hidden_size]를 의미합니다.

우리가 사용한 문장의 배치 사이즈는 1이고, 문장의 길이는 토큰의 길이와 같으므로 7개의 토큰이 현재 문장에 있어 총 길이는 7입니다. 그리고, hidden_size는 표현(임베딩)의 사이즈이므로 BERT-base 모델에서 이 값은 768입니다.

각 토큰의 임베딩을 얻을 수 있습니다:
- last_hidden[0][0]은 문장의 첫번째 토큰인 [CLS] 토큰에 대한 표현입니다.
- last_hidden[0][1]은 문장의 두번째 토큰인 'I' 토큰에 대한 표현입니다.
- last_hidden[0][2]은 문장의 세번째 토큰인 'love' 토큰에 대한 표현입니다.

이와 유사하게, 마지막 인코더 레이어로 부터 모든 토큰에 대한 표현을 얻을 수 있습니다.

다음으로, 선형 변환과 tanh 활성화함수에 의해 후처리된 마지막 인코더 레이어로부터 얻은 [CLS] 토큰에 대한 표현을 함축하는 pooler_output이 있습니다. pooler_output의 사이즈를 출력해봅니다:

In [ ]:
pooler_output.shape

torch.Size([1, 768])

사이즈 [1,768]은 각각 [batch_size, hidden_size]를 의미합니다.

[CLS] 토큰이 문장의 전체적인 표현을 함축하고 있다는 것을 배웠습니다. 그러므로, pooler_output을 주어진 문장인 'I love Paris'의 표현(임베딩)으로 사용할 수 있습니다.

마지막으로, hidden_states는 모든 인코더 레이어의 마지막 층으로부터 얻는 토큰에 대한 표현(임베딩)입니다. 이 반환값은 입력 임베딩 레이어부터 마지막 인코더레이어까지 모든 인코더 레이어(은닉층)의 마지막 레이어의 표현(임베딩)을 갖고 있는 13개의 값으로 이루어진 튜플입니다. 

In [ ]:
len(hidden_states)

13

위에서 알 수 있듯이, 이 반환값은 모든 인코더 레이어로부터 얻은 13개의 토큰 표현값을 갖고 있습니다. 따라서:

- hidden_sttaes[0]는 입력 임베딩 레이어로부터 얻은 모든 토큰의 표현(임베딩)을 포함합니다.
- hidden_sttaes[1]은 첫번째 인코더 레이어로부터 얻은 모든 토큰의 표현(임베딩)을 포함합니다.
- hidden_sttaes[2]은 두번째 인코더 레이어로부터 얻은 모든 토큰의 표현(임베딩)을 포함합니다.

유사하게, hidden_states[12]는 마지막 인코더 레이어로부터 얻은 모든 토큰의 표현(임베딩)을 포함합니다.
이에 대해 더 살펴보겟습니다. 먼저, 입력 임베딩 레이어로부터 얻은 모든 토큰에 대한 표현을 갖고 있는 hidden_states[0]의 사이즈를 출력해봅니다:


In [ ]:
hidden_states[0].shape

torch.Size([1, 7, 768])

사이즈 [1, 7, 768]은 각각 [batch_size, sequence_length, hidden_size]를 의미합니다.

이제, 첫번째 인코더 레이어로부터 얻은 모든 토큰에 대한 표현을 갖고 있는 hidden_states[1]의 사이즈를 출려해봅니다:

In [ ]:
torch.Size([1, 7, 768])

torch.Size([1, 7, 768])

In [ ]:
hidden_states[1].shape

torch.Size([1, 7, 768])

이런 방식으로, 모든 인코더 레이어로부터 얻은 토큰에 대한 표현을 얻을 수 있습니다. 이로써, 사전 학습된 BERT를 사용하여 임베딩을 추출하는 방법을 알아보았는데, sentiment analysis와 같은 다운스트림 작업에서 사전 학습된 BERT를 사용할 수 있을까요? 답은 "Yes!" 입니다. 이에 대해 다음 장에서 배워보도록 합시다